<a href="https://colab.research.google.com/github/gulrukhsorakhmadjanova/5dpro/blob/master/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  Setup and Imports
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import data, filters, transform, feature
from scipy import ndimage
from math import sqrt, pi

print("All libraries imported successfully!")

In [ ]:
# Task 1 - Image Formation & Sampling
print("=== TASK 1: Image Formation & Sampling ===")

# Load high-resolution image
image_hr = data.camera()
print(f"Original image shape: {image_hr.shape}")

downsample_factors = [2, 4, 8]

# Create subplots for visualization
fig, axes = plt.subplots(2, len(downsample_factors) + 1, figsize=(16, 8))

# Display original image
axes[0, 0].imshow(image_hr, cmap='gray')
axes[0, 0].set_title('Original High-Res Image')
axes[0, 0].axis('off')

axes[1, 0].imshow(image_hr, cmap='gray')
axes[1, 0].set_title('Original High-Res Image')
axes[1, 0].axis('off')

for i, factor in enumerate(downsample_factors):
    # Naive Downsampling (no prefiltering)
    img_naive = image_hr[::factor, ::factor]

    # Prefiltering with Gaussian Blur before downsampling
    sigma = factor / 2  # Common heuristic
    img_blurred = filters.gaussian(image_hr, sigma=sigma)
    img_prefiltered = img_blurred[::factor, ::factor]

    # Plotting
    axes[0, i + 1].imshow(img_naive, cmap='gray')
    axes[0, i + 1].set_title(f'Naive Downsample\n(Factor {factor})')
    axes[0, i + 1].axis('off')

    axes[1, i + 1].imshow(img_prefiltered, cmap='gray')
    axes[1, i + 1].set_title(f'Prefiltered Downsample\n(Factor {factor}, σ={sigma})')
    axes[1, i + 1].axis('off')

plt.tight_layout()
plt.show()

# Frequency spectra analysis
fig, axes = plt.subplots(2, len(downsample_factors) + 1, figsize=(16, 8))

# Original image FFT
fft_original = np.fft.fftshift(np.fft.fft2(image_hr))
axes[0, 0].imshow(np.log1p(np.abs(fft_original)), cmap='viridis')
axes[0, 0].set_title('Original FFT Spectrum')
axes[0, 0].axis('off')

axes[1, 0].imshow(np.log1p(np.abs(fft_original)), cmap='viridis')
axes[1, 0].set_title('Original FFT Spectrum')
axes[1, 0].axis('off')

for i, factor in enumerate(downsample_factors):
    # Naive downsampled FFT
    img_naive = image_hr[::factor, ::factor]
    fft_naive = np.fft.fftshift(np.fft.fft2(img_naive))

    # Prefiltered FFT
    sigma = factor / 2
    img_blurred = filters.gaussian(image_hr, sigma=sigma)
    img_prefiltered = img_blurred[::factor, ::factor]
    fft_prefiltered = np.fft.fftshift(np.fft.fft2(img_prefiltered))

    axes[0, i + 1].imshow(np.log1p(np.abs(fft_naive)), cmap='viridis')
    axes[0, i + 1].set_title(f'Naive FFT (Factor {factor})')
    axes[0, i + 1].axis('off')

    axes[1, i + 1].imshow(np.log1p(np.abs(fft_prefiltered)), cmap='viridis')
    axes[1, i + 1].set_title(f'Prefiltered FFT (Factor {factor})')
    axes[1, i + 1].axis('off')

plt.tight_layout()
plt.show()

print("\nExplanation of Prefiltering and Aliasing:")
print("""
Aliasing occurs when high-frequency components in the original signal fold back into lower frequencies
due to insufficient sampling rate (violating the Nyquist criterion). Prefiltering applies a low-pass
filter (Gaussian blur) before sampling to attenuate frequencies above the Nyquist limit (half the
sampling rate). This prevents high frequencies from masquerading as lower frequencies, eliminating
aliasing artifacts like moiré patterns and jagged edges. The Gaussian kernel's σ is typically chosen
proportional to the scale factor to properly band-limit the signal.
""")

In [ ]:
#  Task 2 - Linear Filters & Convolution (Manual Implementation)
print("=== TASK 2: Linear Filters & Convolution ===")

def manual_conv2d(image, kernel):
    """Manual 2D convolution function with zero-padding."""
    k_h, k_w = kernel.shape
    pad_h, pad_w = k_h // 2, k_w // 2

    # Add zero-padding
    image_padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant')
    output = np.zeros_like(image, dtype=float)

    # Perform convolution
    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            region = image_padded[i:i + k_h, j:j + k_w]
            output[i, j] = np.sum(region * kernel)
    return output

# Define Box filters
box_3x3 = np.ones((3, 3)) / 9.0
box_5x5 = np.ones((5, 5)) / 25.0

# Create Gaussian kernel function
def gaussian_kernel(size, sigma=1.0):
    """Create a 2D Gaussian kernel."""
    ax = np.linspace(-(size // 2), size // 2, size)
    x, y = np.meshgrid(ax, ax)
    kernel = np.exp(-(x**2 + y**2) / (2.0 * sigma**2))
    return kernel / np.sum(kernel)

# Generate Gaussian kernels
gauss_3x3_s1 = gaussian_kernel(3, sigma=1.0)
gauss_5x5_s2 = gaussian_kernel(5, sigma=2.0)

print("Box 3x3 kernel:")
print(box_3x3)
print("\nGaussian 3x3 (σ=1) kernel:")
print(gauss_3x3_s1)

In [ ]:
#  Task 2 - Filter Application and Comparison
# Apply all filters manually and with OpenCV
filters_to_test = [
    ('Box 3x3', box_3x3),
    ('Box 5x5', box_5x5),
    ('Gaussian 3x3 σ=1', gauss_3x3_s1),
    ('Gaussian 5x5 σ=2', gauss_5x5_s2)
]

# Convert image to float for processing
image_float = image_hr.astype(float)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx, (name, kernel) in enumerate(filters_to_test):
    # Manual convolution
    manual_result = manual_conv2d(image_float, kernel)

    # OpenCV convolution
    opencv_result = cv2.filter2D(image_float, -1, kernel)

    # Calculate difference
    diff = np.max(np.abs(manual_result - opencv_result))
    print(f"{name} - Max difference: {diff:.2e}")

    # Visualization
    axes[0, idx].imshow(manual_result, cmap='gray')
    axes[0, idx].set_title(f'Manual: {name}')
    axes[0, idx].axis('off')

    axes[1, idx].imshow(opencv_result, cmap='gray')
    axes[1, idx].set_title(f'OpenCV: {name}')
    axes[1, idx].axis('off')

plt.tight_layout()
plt.show()

print("\nTrade-off Analysis:")
print("""
Smoothing vs. Edge Sharpness Trade-off:
- Larger kernels (5x5 vs 3x3) provide more aggressive noise reduction but cause greater edge blurring
- Gaussian filters preserve edge quality better than box filters of equivalent size due to their
  distance-weighted kernel, resulting in more natural transitions
- Box 5x5: Strong smoothing but significant edge blurring
- Gaussian 5x5 σ=2: Similar smoothing level but better edge preservation
- Choice depends on application: noise reduction vs. feature preservation
""")

In [ ]:
#  Task 2 - Bonus: Convolution Commutativity
print("=== BONUS: Convolution Commutativity ===")

# Test commutativity: A * B = B * A
kernel_a = np.array([[1, 2], [3, 4]])
kernel_b = np.array([[0, 1], [1, 0]])

# Use a small test patch from the image
test_patch = image_hr[100:102, 100:102].astype(float)
print("Test patch:")
print(test_patch)

# Apply A then B
result_AB = manual_conv2d(manual_conv2d(test_patch, kernel_a), kernel_b)

# Apply B then A
result_BA = manual_conv2d(manual_conv2d(test_patch, kernel_b), kernel_a)

print(f"\nResult of (A * B):\n{result_AB}")
print(f"\nResult of (B * A):\n{result_BA}")
print(f"\nMax absolute difference: {np.max(np.abs(result_AB - result_BA))}")

print("\nCommutativity confirmed: A * B = B * A")

In [ ]:
#  Task 3 - Edge Detection (Sobel and Prewitt Implementation)
print("=== TASK 3: Edge Detection ===")

# Define Sobel and Prewitt kernels
sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

prewitt_x = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
prewitt_y = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])

print("Sobel X kernel:")
print(sobel_x)
print("\nSobel Y kernel:")
print(sobel_y)
print("\nPrewitt X kernel:")
print(prewitt_x)
print("\nPrewitt Y kernel:")
print(prewitt_y)

def compute_gradient_magnitude_orientation(img, kx, ky, name):
    """Compute gradient magnitude and orientation for given kernels."""
    Gx = cv2.filter2D(img.astype(float), -1, kx)
    Gy = cv2.filter2D(img.astype(float), -1, ky)

    magnitude = np.sqrt(Gx**2 + Gy**2)
    orientation = np.arctan2(Gy, Gx)  # in radians

    # Normalize magnitude for display
    magnitude_display = (magnitude / magnitude.max() * 255).astype(np.uint8)

    return Gx, Gy, magnitude, orientation, magnitude_display

# Compute for Sobel
Gx_sobel, Gy_sobel, mag_sobel, orient_sobel, mag_disp_sobel = compute_gradient_magnitude_orientation(
    image_hr, sobel_x, sobel_y, "Sobel")

# Compute for Prewitt
Gx_prewitt, Gy_prewitt, mag_prewitt, orient_prewitt, mag_disp_prewitt = compute_gradient_magnitude_orientation(
    image_hr, prewitt_x, prewitt_y, "Prewitt")

In [ ]:
#  Task 3 - Edge Detection Visualization and Comparison
# Visualize Sobel results
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Sobel results
axes[0, 0].imshow(Gx_sobel, cmap='seismic')
axes[0, 0].set_title('Sobel - Gx (Horizontal)')
axes[0, 0].axis('off')

axes[0, 1].imshow(Gy_sobel, cmap='seismic')
axes[0, 1].set_title('Sobel - Gy (Vertical)')
axes[0, 1].axis('off')

axes[0, 2].imshow(mag_disp_sobel, cmap='gray')
axes[0, 2].set_title('Sobel - Gradient Magnitude')
axes[0, 2].axis('off')

axes[0, 3].imshow(orient_sobel, cmap='hsv')
axes[0, 3].set_title('Sobel - Orientation')
axes[0, 3].axis('off')

# Prewitt results
axes[1, 0].imshow(Gx_prewitt, cmap='seismic')
axes[1, 0].set_title('Prewitt - Gx (Horizontal)')
axes[1, 0].axis('off')

axes[1, 1].imshow(Gy_prewitt, cmap='seismic')
axes[1, 1].set_title('Prewitt - Gy (Vertical)')
axes[1, 1].axis('off')

axes[1, 2].imshow(mag_disp_prewitt, cmap='gray')
axes[1, 2].set_title('Prewitt - Gradient Magnitude')
axes[1, 2].axis('off')

axes[1, 3].imshow(orient_prewitt, cmap='hsv')
axes[1, 3].set_title('Prewitt - Orientation')
axes[1, 3].axis('off')

plt.tight_layout()
plt.show()

# Compare with OpenCV built-in Sobel
opencv_sobelx = cv2.Sobel(image_hr, cv2.CV_64F, 1, 0, ksize=3)
opencv_sobely = cv2.Sobel(image_hr, cv2.CV_64F, 0, 1, ksize=3)
opencv_mag = np.sqrt(opencv_sobelx**2 + opencv_sobely**2)

# Normalize for comparison
opencv_mag_display = (opencv_mag / opencv_mag.max() * 255).astype(np.uint8)

print("Comparison with OpenCV Sobel:")
print(f"Max difference in Gx: {np.max(np.abs(Gx_sobel - opencv_sobelx)):.2e}")
print(f"Max difference in Gy: {np.max(np.abs(Gy_sobel - opencv_sobely)):.2e}")

print("\nGradient Direction vs Edge Orientation Relationship:")
print("""
The gradient vector (Gx, Gy) points in the direction of the steepest intensity increase.
Edge orientation is PERPENDICULAR to the gradient direction because edges represent
boundaries between regions of different intensity.

- Gradient Direction: θ = arctan2(Gy, Gx)
- Edge Orientation: θ_edge = θ + 90°

Thus, if the gradient points uphill across an edge, the edge itself runs perpendicular
to this direction.
""")

In [ ]:
#  Task 3 - Bonus: Canny Edge Detector
print("=== BONUS: Canny Edge Detector ===")

# Apply Canny with different thresholds
edges_canny_low = cv2.Canny(image_hr, 50, 150)
edges_canny_medium = cv2.Canny(image_hr, 100, 200)
edges_canny_high = cv2.Canny(image_hr, 150, 250)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Original image
axes[0, 0].imshow(image_hr, cmap='gray')
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

# Canny with different thresholds
axes[0, 1].imshow(edges_canny_low, cmap='gray')
axes[0, 1].set_title('Canny (Low: 50, 150)\nMore edges, more noise')
axes[0, 1].axis('off')

axes[1, 0].imshow(edges_canny_medium, cmap='gray')
axes[1, 0].set_title('Canny (Medium: 100, 200)\nBalanced')
axes[1, 0].axis('off')

axes[1, 1].imshow(edges_canny_high, cmap='gray')
axes[1, 1].set_title('Canny (High: 150, 250)\nCleaner, fewer edges')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("Canny Detector Analysis:")
print("""
The Canny detector uses hysteresis thresholding with two thresholds:
- Low threshold: Identifies potential edge pixels (including weak edges)
- High threshold: Identifies strong, definite edge pixels

Analysis:
- Low thresholds (50, 150): Detect more edges including weak ones, but also more noise
- Medium thresholds (100, 200): Good balance between edge completeness and noise reduction
- High thresholds (150, 250): Only strongest edges remain, very clean but may miss details

Hysteresis tracking connects weak edges to strong ones, providing better continuity
than simple thresholding while maintaining noise robustness.
""")

In [ ]:
#  Task 4 - Geometric Transformations
print("=== TASK 4: Geometric Transformations ===")

h, w = image_hr.shape
print(f"Image dimensions: {h} x {w}")

# 1. Translation (Shift 50px right, 30px down)
M_translate = np.float32([[1, 0, 50], [0, 1, 30]])
img_translated = cv2.warpAffine(image_hr, M_translate, (w, h))

# 2. Rotation (30 degrees around image center)
center = (w // 2, h // 2)
angle = 30
scale = 1.0
M_rotate = cv2.getRotationMatrix2D(center, angle, scale)
img_rotated = cv2.warpAffine(image_hr, M_rotate, (w, h))

# 3. Scaling (Scale down to 80% around center)
scale_factor = 0.8
M_scale = cv2.getRotationMatrix2D(center, 0, scale_factor)
img_scaled = cv2.warpAffine(image_hr, M_scale, (w, h))

# 4. Perspective Warp
src_pts = np.float32([[50, 50], [w-50, 50], [50, h-50], [w-50, h-50]])
dst_pts = np.float32([[0, 0], [w, 100], [0, h], [w, h-100]])
M_perspective = cv2.getPerspectiveTransform(src_pts, dst_pts)
img_perspective = cv2.warpPerspective(image_hr, M_perspective, (w, h))

print("Translation Matrix:")
print(M_translate)
print(f"\nRotation Matrix (θ={angle}°):")
print(M_rotate)
print(f"\nScaling Matrix (scale={scale_factor}):")
print(M_scale)
print("\nPerspective Transformation Matrix:")
print(M_perspective)

In [ ]:
#  Task 4 - Transformation Visualization and Analysis
# Visualize all transformations
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

transformations = [
    ('Translation\n(50px right, 30px down)', img_translated),
    ('Rotation\n(30° around center)', img_rotated),
    ('Scaling\n(80% size)', img_scaled),
    ('Perspective Warp', img_perspective)
]

for idx, (title, img) in enumerate(transformations):
    ax = axes[idx // 2, idx % 2]
    ax.imshow(img, cmap='gray')
    ax.set_title(title)
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Transformation Matrix Analysis:")
print("""
AFFINE TRANSFORMATIONS (warpAffine):
- Preserve parallelism and ratios of areas
- Matrix form: M = [[a, b, c], [d, e, f]]
  - c, f: Translation components (tx, ty)
  - [[a,b],[d,e]]: Linear transformation (rotation, scaling, shearing)
  - Rotation: a=e=cos(θ), b=-sin(θ), d=sin(θ)
  - Scaling: a=sx, e=sy (diagonal elements)

PERSPECTIVE TRANSFORMATION (warpPerspective):
- Does NOT preserve parallelism (simulates 3D perspective)
- 3x3 matrix with bottom row [v1, v2, v3] for projective effects
- Creates vanishing points where parallel lines converge
- More general than affine, can represent all affine + projective transforms

Key Difference: Affine preserves parallel lines, perspective does not.
""")

In [ ]:
#  Final Reflection and Summary
print("=== REFLECTION & SUMMARY ===")

print("""
KEY INSIGHTS FROM THIS ASSIGNMENT:

1. SAMPLING THEORY IS PRACTICAL: Task 1 made the Nyquist-Shannon theorem tangible.
   Seeing severe aliasing in naïve downsampling versus clean prefiltered results
   underscores that sampling requires careful frequency management.

2. FILTERS INVOLVE TRADE-OFFS: Manual convolution implementation revealed the
   computational nature of filtering. The Box vs Gaussian comparison showed that
   stronger smoothing inherently compromises edge integrity - there's no free lunch.

3. EDGES ARE FUNDAMENTAL FEATURES: Implementing Sobel/Prewitt demonstrated that
   edge detection is more than kernel application. Understanding the perpendicular
   relationship between gradient direction and edge orientation is crucial for
   higher-level vision tasks.

4. GEOMETRY HAS MATHEMATICAL ROOTS: The transformation exercises connected linear
   algebra to visual outcomes. The distinction between affine (parallel-preserving)
   and perspective (vanishing points) transformations provides the foundation for
   image registration and 3D computer vision.

REPRODUCIBILITY NOTE:
All code is self-contained and uses standard computer vision libraries. The manual
implementations (convolution, edge detection) verify our understanding of the
underlying operations, while OpenCV comparisons ensure practical correctness.

This module successfully bridges mathematical theory and practical implementation,
providing the essential toolkit for advanced computer vision topics.
""")
